# Import Libraries

In [1]:
import os
import sys
import time
import math
import pickle
import random
import pathlib
import sqlite3
import tempfile
import importlib
import subprocess

import numpy as np
import pandas as pd
import seaborn as sn
from tqdm import tqdm
import matplotlib.pyplot as plt
from configparser import ConfigParser
from distutils.spawn import find_executable
from collections import OrderedDict, namedtuple
from mlxtend.feature_selection import ColumnSelector

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
import torch.nn.functional as F
from torch.autograd import Variable
from sru import SRU, SRUCell

# Set Parameters

In [2]:
np.random.seed(0)

torch.cuda.empty_cache()
torch.manual_seed(0)
torch.backends.cudnn.benchmark=True

os.environ['CUDA_VISIBLE_DEVICES'] = '1' # GPU Number
os.environ['TORCH_EXTENSIONS_DIR'] = os.popen('pwd').read().strip('\n') # SRU Temp Folder

In [3]:
# Mean and Standard Deviation for Noise - used for SNR analysis
scr5_mean = 33.17964
scr5_std = 6.6672482

In [4]:
NUM_EPOCHS = 10 # No Of Training Epochs
TRAIN_BATCH_SIZE = 1024 # Batch size for Training
VAL_BATCH_SIZE = 128 # Batch size for Validation and Testing

# Parameters for RNN based Architectures
EMBEDDING_DIM = 3
HIDDEN_DIM = 100
TAGSET_SIZE = 2

# DNN Modules

In [5]:
class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.selu = nn.SELU()
        self.drop = nn.AlphaDropout(p=0.5)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=16, kernel_size=3, padding=1)
        self.linear = nn.Linear(in_features=16*3, out_features=2)

    def forward(self, input):
        output = self.conv1(input)
        output = self.selu(output)
        output = self.conv2(output)
        output = self.selu(output)
        output = output.view(-1,16*3)
        output = self.drop(output)
        output = self.linear(output)
        return output
    
    

In [6]:
class GRUTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, tagset_size):
        super(GRUTagger, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.gru = nn.GRU(embedding_dim, hidden_dim)
        
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        
    
    def forward(self, sentence):
        gru_out, _ = self.gru(sentence.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(gru_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores
    

In [7]:
class BGRUTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, tagset_size):
        super(BGRUTagger, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.bgru = nn.GRU(embedding_dim, hidden_dim,bidirectional=True)
        
        self.hidden2tag = nn.Linear(hidden_dim*2, tagset_size)
        
    
    def forward(self, sentence):
        gru_out, _ = self.bgru(sentence.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(gru_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores
    

In [8]:
class BSRUTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, tagset_size):
        super(BSRUTagger, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.bsru = SRU(embedding_dim, hidden_dim,2,bidirectional=True)
        
        self.hidden2tag = nn.Linear(hidden_dim*2, tagset_size)
        
    
    def forward(self, sentence):
        sru_out, _ = self.bsru(sentence.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(sru_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores
    

In [9]:
class CBSDNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        nn.init.xavier_normal_(self.conv1.weight)

        self.conv2 = nn.Conv1d(in_channels=16, out_channels=16, kernel_size=3, padding=1)
        nn.init.xavier_normal_(self.conv2.weight)

        self.linear1 = nn.Linear(16*3, 100)
        
        self.bsru = SRU(input_size=100, hidden_size=100,num_layers=2,bidirectional=True)
        self.linear2 = nn.Linear(100*2, 100*2)
        nn.init.xavier_normal_(self.linear2.weight)

        self.linear3 = nn.Linear(100*2, 2)

        self.relu = nn.ReLU()

        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.2)

        self.softmax = nn.Softmax(dim=1)

    def forward(self, input):
        
        output = self.conv1(input)
        output = self.relu(output)

        output = self.conv2(output)
        output = self.relu(output)
        output = self.dropout1(output)
                
        output = output.view(-1, 16*3)
        output1 = self.linear1(output)

        output = output1.view(len(output1),1 ,-1 )
        output, _ = self.bsru(output)
        
        output = output.view(len(output1), -1)

        output = self.linear2(output)
        output = self.relu(output)

        output = self.dropout2(output)

        output = self.linear3(output)

        output = F.log_softmax(output, dim=1)
        return output


# Acquire and Prepare Data

In [10]:
with open('../ssp_cldnn/scrimmage5_link_dataset.pickle', 'rb') as file:
    link_dataset = pickle.load(file)

In [11]:
# Retain only the features selected by the RFE method
# Columns : 0 - SNR, 1 - MCS, 2 - Center Frequency, 3 - Bandwidth, [4:19] - PSD

cols = torch.LongTensor([0,1,3]) 

link_data = [(link_datas[0][:,cols], link_datas[1]) for link_datas in link_dataset] 

In [12]:
# Without Pilot

# Do a 40:10:50 train:validation:test split 
train_data = link_data[:251]
val_data = link_data[251:314]
test_data = link_data[314:]

train_x = torch.cat(tuple(link[0] for link in train_data),dim=0)
train_y = torch.cat(tuple(link[1] for link in train_data),dim=0)

val_x = torch.cat(tuple(link[0] for link in val_data),dim=0)
val_y = torch.cat(tuple(link[1] for link in val_data),dim=0)

test_x = torch.cat(tuple(link[0] for link in test_data),dim=0)
test_y = torch.cat(tuple(link[1] for link in test_data),dim=0)

# Reshape data for CNN and CBSDNN
train_x_cnn = train_x.view(-1,1,3)
val_x_cnn = val_x.view(-1,1,3)
test_x_cnn = test_x.view(-1,1,3)

In [13]:
# With Pilot

datadist = [0.4, 0.1, 0.5] # distribution among train, test and validation

train_x_npn = []
train_y_npn = []
val_x_npn = []
val_y_npn = []
test_x_npn = []
test_y_npn = []

for i in range(len(link_data)):
    datalen = len(link_data[i][1])
    trainlen = int(datalen*datadist[0])
    vallen = int(datalen*(datadist[0]+datadist[1]))

    train_x_npn.append(link_data[i][0][0:trainlen].numpy())    
    train_y_npn.append(link_data[i][1][0:trainlen].numpy())
    
    val_x_npn.append(link_data[i][0][trainlen:vallen].numpy())    
    val_y_npn.append(link_data[i][1][trainlen:vallen].numpy())
    
    test_x_npn.append(link_data[i][0][vallen:datalen].numpy())
    test_y_npn.append(link_data[i][1][vallen:datalen].numpy())

train_x_np = np.concatenate(train_x_npn)
train_y_np = np.concatenate(train_y_npn)
val_x_np = np.concatenate(val_x_npn)
val_y_np = np.concatenate(val_y_npn)
test_x_np = np.concatenate(test_x_npn)
test_y_np = np.concatenate(test_y_npn)

train_x_plt = torch.from_numpy(train_x_np)
val_x_plt = torch.from_numpy(val_x_np)
test_x_plt = torch.from_numpy(test_x_np)
train_y_plt = torch.from_numpy(train_y_np)
val_y_plt = torch.from_numpy(val_y_np)
test_y_plt = torch.from_numpy(test_y_np)

# Reshape data for CNN and CBSDNN
train_x_plt_cnn = train_x_plt.view(-1,1,3)
val_x_plt_cnn = val_x_plt.view(-1,1,3)
test_x_plt_cnn = test_x_plt.view(-1,1,3)

# Running Tests

In [14]:
# Initialize Result Variables
cnn_scr5_res = {}
gru_scr5_res = {}
bgru_scr5_res = {}
bsru_scr5_res = {}
cbsdnn_scr5_res = {}

# Without Pilot

# CNN

In [15]:
# Train

model = CNN()
model.cuda()

loss_function = nn.CrossEntropyLoss(weight=torch.tensor([1.0,1.0])).cuda()
optimizer = optim.SGD(model.parameters(), lr = 0.01)

train_dataloader=data.DataLoader(data.TensorDataset(train_x_cnn,train_y),batch_size=TRAIN_BATCH_SIZE, 
                                   shuffle=True, num_workers=16, pin_memory=True)

val_dataloader=data.DataLoader(data.TensorDataset(val_x_cnn,val_y),batch_size=VAL_BATCH_SIZE, 
                                 shuffle=False, num_workers=16, pin_memory=True)

test_dataloader=data.DataLoader(data.TensorDataset(test_x_cnn,test_y), batch_size=VAL_BATCH_SIZE,
                                  shuffle=False, num_workers=16, pin_memory=True)

model.train()

for epoch_idx in range(NUM_EPOCHS):  # again, normally you would NOT do 300 epochs, it is toy data

    progress_training_epoch = tqdm(
        train_dataloader, 
        desc=f'Epoch {epoch_idx + 1}/{NUM_EPOCHS}, Training',
        miniters=1, ncols=88, position=0,
        leave=True, total=len(train_dataloader), smoothing=.9)

    progress_validation_epoch = tqdm(
        val_dataloader, 
        desc=f'Epoch {epoch_idx + 1}/{NUM_EPOCHS}, Validation',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(val_dataloader), smoothing=.9)

    train_loss = 0
    train_size = 0
    for idx, (input, target) in enumerate(progress_training_epoch):
        input = input.cuda()
        target = target.cuda()
        model.zero_grad()
        predict = model(input)
        loss = loss_function(predict, target)
        loss.backward()
        optimizer.step()
        train_loss += loss * target.size()[0]
        train_size += target.size()[0]

    test_loss = 0
    test_size = 0
    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (batch_input, batch_target) in enumerate(progress_validation_epoch):
            batch_input = batch_input.cuda()
            batch_target = batch_target.cuda()
            batch_predict = model(batch_input)
            loss = loss_function(batch_predict, batch_target)
            predict.append(batch_predict.argmax(dim=1).cpu().numpy())
            target.append(batch_target.cpu().numpy())        
            test_loss += loss * batch_target.size()[0]
            test_size += batch_target.size()[0]
    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)
   
    print(f'train loss:{train_loss.item()/train_size: .5f}, '
          f'validation loss:{test_loss.item()/test_size: .5f}')

Epoch 2/10, Validation:   0%|                                  | 0/9128 [00:00<?, ?it/s]

train loss: 0.53788, validation loss: 0.61614


Epoch 3/10, Validation:   0%|                                  | 0/9128 [00:00<?, ?it/s]

train loss: 0.51292, validation loss: 0.56793


Epoch 4/10, Validation:   0%|                                  | 0/9128 [00:00<?, ?it/s]

...

train loss: 0.50236, validation loss: 0.55974


Epoch 10/10, Validation:   0%|                                 | 0/9128 [00:00<?, ?it/s]

train loss: 0.50126, validation loss: 0.55836


Epoch 10/10, Validation: 100%|█████████████████████| 9128/9128 [00:18<00:00, 483.97it/s]


train loss: 0.50053, validation loss: 0.55715


In [16]:
# Test

progress_test_epoch = tqdm(
    test_dataloader, 
    desc=f'Epoch {epoch_idx + 1}/{NUM_EPOCHS}, Test',
    miniters=1, ncols=88, position=0, 
    leave=True, total=len(test_dataloader), smoothing=.9)

predict = [] # Predicted Label
target = [] # Target Lable
snrss = [] # SNR (Calculated from Noise Variance)
model.eval()
with torch.no_grad():
    for idx, (batch_input, batch_target) in enumerate(progress_test_epoch):
        batch_input = batch_input.cuda()
        batch_target = batch_target.cuda()
        batch_predict = model(batch_input)
        predict.append(batch_predict.argmax(dim=1).cpu().numpy())
        target.append(batch_target.cpu().numpy())        
        snrss.append(batch_input.cpu().numpy()[:,0][:,0])
predict = np.concatenate(predict, axis=0)
target = np.concatenate(target, axis=0)
snrss = np.concatenate(snrss, axis=0)

cnn_scr5_res['cnn_scr5_prd'] = predict
cnn_scr5_res['cnn_scr5_trg'] = target
cnn_scr5_res['cnn_scr5_snr'] = snrss * scr5_std + scr5_mean

Epoch 10/10, Test: 100%|████████████████████████| 45899/45899 [00:35<00:00, 1308.28it/s]


# GRU

In [17]:
# Train

model = GRUTagger(EMBEDDING_DIM, HIDDEN_DIM, TAGSET_SIZE).cuda()
model.cuda()
loss_function = nn.NLLLoss(weight=torch.tensor([1.0,1.0])).cuda()
optimizer = optim.Adam(model.parameters(), 0.0001)

train_dataloader=data.DataLoader(data.TensorDataset(train_x,train_y),batch_size=TRAIN_BATCH_SIZE, 
                                   shuffle=False, num_workers=16, pin_memory=True)

val_dataloader=data.DataLoader(data.TensorDataset(val_x,val_y),batch_size=VAL_BATCH_SIZE, 
                                 shuffle=False, num_workers=16, pin_memory=True)

test_dataloader=data.DataLoader(data.TensorDataset(test_x,test_y), batch_size=VAL_BATCH_SIZE,
                                  shuffle=False, num_workers=16, pin_memory=True)

for epoch_idx in range(NUM_EPOCHS): 

    progress_training_epoch = tqdm(
        train_dataloader, 
        desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Training',
        miniters=1, ncols=88, position=0,
        leave=True, total=len(train_dataloader), smoothing=.9)

    progress_validation_epoch = tqdm(
        val_dataloader, 
        desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Validation',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(val_dataloader), smoothing=.9)

    train_loss = 0
    train_size = 0
    model.train()
    for idx, (sentence, tags) in enumerate(progress_training_epoch):
        sentence = sentence.cuda()
        tags = tags.cuda()
        model.zero_grad()
        tag_scores = model(sentence)
        loss = loss_function(tag_scores, tags)
        loss.backward()
        optimizer.step()
        train_loss += loss * tags.size()[0]
        train_size += tags.size()[0]

    test_loss = 0
    test_size = 0
    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (sentence, tags) in enumerate(progress_validation_epoch):
            sentence = sentence.cuda()
            tags = tags.cuda()
            tag_scores = model(sentence)
            loss = loss_function(tag_scores, tags)
            predict.append(tag_scores.argmax(dim=1).cpu().numpy())
            target.append(tags.cpu().numpy())        
            test_loss += loss * tags.size()[0]
            test_size += tags.size()[0]
    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)

    print(f'train loss:{train_loss.item()/train_size: .5f}, '
          f'validation loss:{test_loss.item()/test_size: .5f}')

Epoch 2/10, Validation:   0%|                                  | 0/9128 [00:00<?, ?it/s]

train loss: 0.51048, validation loss: 0.54652


Epoch 3/10, Validation:   0%|                                  | 0/9128 [00:00<?, ?it/s]

train loss: 0.49632, validation loss: 0.54348


Epoch 4/10, Validation:   0%|                                  | 0/9128 [00:00<?, ?it/s]

...

train loss: 0.48160, validation loss: 0.53381


Epoch 10/10, Validation:   0%|                                 | 0/9128 [00:00<?, ?it/s]

train loss: 0.47457, validation loss: 0.54283


Epoch 10/10, Validation: 100%|██████████████████████| 9128/9128 [02:01<00:00, 74.98it/s]


train loss: 0.46676, validation loss: 0.54178


In [18]:
# Test

progress_test_epoch = tqdm(
    test_dataloader, 
    desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Test',
    miniters=1, ncols=88, position=0, 
    leave=True, total=len(test_dataloader), smoothing=.9)

predict = []
target = []
snrss = []
model.eval()
with torch.no_grad():
    for idx, (sentence, tags) in enumerate(progress_test_epoch):
        sentence = sentence.cuda()
        tags = tags.cuda()
        tag_scores = model(sentence)
        predict.append(tag_scores.argmax(dim=1).cpu().numpy())
        target.append(tags.cpu().numpy())
        snrss.append(sentence.cpu().numpy()[:,0])
        
predict = np.concatenate(predict, axis=0)
target = np.concatenate(target, axis=0)
snrss = np.concatenate(snrss, axis=0)

gru_scr5_res['gru_scr5_prd'] = predict
gru_scr5_res['gru_scr5_trg'] = target
gru_scr5_res['gru_scr5_snr'] = snrss * scr5_std + scr5_mean

Epoch 10/10, Test: 100%|█████████████████████████| 45899/45899 [01:27<00:00, 524.95it/s]


# BGRU

In [ ]:
# Train

model = BGRUTagger(EMBEDDING_DIM, HIDDEN_DIM, TAGSET_SIZE).cuda()
model.cuda()
loss_function = nn.NLLLoss(weight=torch.tensor([1.0,1.0])).cuda()
optimizer = optim.Adam(model.parameters(), 0.0001)

train_dataloader=data.DataLoader(data.TensorDataset(train_x,train_y),batch_size=TRAIN_BATCH_SIZE, 
                                   shuffle=False, num_workers=16, pin_memory=True)

val_dataloader=data.DataLoader(data.TensorDataset(val_x,val_y),batch_size=VAL_BATCH_SIZE, 
                                 shuffle=False, num_workers=16, pin_memory=True)

test_dataloader=data.DataLoader(data.TensorDataset(test_x,test_y), batch_size=VAL_BATCH_SIZE,
                                  shuffle=False, num_workers=16, pin_memory=True)

for epoch_idx in range(NUM_EPOCHS): 

    progress_training_epoch = tqdm(
        train_dataloader, 
        desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Training',
        miniters=1, ncols=88, position=0,
        leave=True, total=len(train_dataloader), smoothing=.9)

    progress_validation_epoch = tqdm(
        val_dataloader, 
        desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Validation',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(val_dataloader), smoothing=.9)

    train_loss = 0
    train_size = 0
    model.train()
    for idx, (sentence, tags) in enumerate(progress_training_epoch):
        sentence = sentence.cuda()
        tags = tags.cuda()
        model.zero_grad()
        tag_scores = model(sentence)
        loss = loss_function(tag_scores, tags)
        loss.backward()
        optimizer.step()
        train_loss += loss * tags.size()[0]
        train_size += tags.size()[0]

    test_loss = 0
    test_size = 0
    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (sentence, tags) in enumerate(progress_validation_epoch):
            sentence = sentence.cuda()
            tags = tags.cuda()
            tag_scores = model(sentence)
            loss = loss_function(tag_scores, tags)
            predict.append(tag_scores.argmax(dim=1).cpu().numpy())
            target.append(tags.cpu().numpy())        
            test_loss += loss * tags.size()[0]
            test_size += tags.size()[0]
    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)

    print(f'train loss:{train_loss.item()/train_size: .5f}, '
          f'validation loss:{test_loss.item()/test_size: .5f}')

Epoch 2/10, Validation:   0%|                                  | 0/9128 [00:00<?, ?it/s]

train loss: 0.47320, validation loss: 0.49137


Epoch 3/10, Validation:   0%|                                  | 0/9128 [00:00<?, ?it/s]

train loss: 0.42092, validation loss: 0.47439


Epoch 4/10, Validation:   0%|                                  | 0/9128 [00:00<?, ?it/s]

...

Epoch 8/10, Validation:   0%|                                  | 0/9128 [00:00<?, ?it/s]

train loss: 0.40317, validation loss: 0.46803


Epoch 9/10, Validation:   0%|                                  | 0/9128 [00:00<?, ?it/s]

train loss: 0.39891, validation loss: 0.47492


Epoch 9/10, Training:  82%|████████████████████▌    | 3724/4518 [02:39<00:32, 24.23it/s]

In [ ]:
# Test

progress_test_epoch = tqdm(
    test_dataloader, 
    desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Test',
    miniters=1, ncols=88, position=0, 
    leave=True, total=len(test_dataloader), smoothing=.9)

predict = []
target = []
snrss = []
model.eval()
with torch.no_grad():
    for idx, (sentence, tags) in enumerate(progress_test_epoch):
        sentence = sentence.cuda()
        tags = tags.cuda()
        tag_scores = model(sentence)
        predict.append(tag_scores.argmax(dim=1).cpu().numpy())
        target.append(tags.cpu().numpy())
        snrss.append(sentence.cpu().numpy()[:,0])
        
predict = np.concatenate(predict, axis=0)
target = np.concatenate(target, axis=0)
snrss = np.concatenate(snrss, axis=0)

bgru_scr5_res['bgru_scr5_prd'] = predict
bgru_scr5_res['bgru_scr5_trg'] = target
bgru_scr5_res['bgru_scr5_snr'] = snrss * scr5_std + scr5_mean

# BSRU

In [ ]:
# Train

model = BSRUTagger(EMBEDDING_DIM, HIDDEN_DIM, TAGSET_SIZE).cuda()
model.cuda()
loss_function = nn.NLLLoss(weight=torch.tensor([1.0,1.0])).cuda()
optimizer = optim.Adam(model.parameters(), 0.0001)

train_dataloader=data.DataLoader(data.TensorDataset(train_x,train_y),batch_size=TRAIN_BATCH_SIZE, 
                                   shuffle=False, num_workers=16, pin_memory=True)

val_dataloader=data.DataLoader(data.TensorDataset(val_x,val_y),batch_size=VAL_BATCH_SIZE, 
                                 shuffle=False, num_workers=16, pin_memory=True)

test_dataloader=data.DataLoader(data.TensorDataset(test_x,test_y), batch_size=VAL_BATCH_SIZE,
                                  shuffle=False, num_workers=16, pin_memory=True)

for epoch_idx in range(NUM_EPOCHS): 

    progress_training_epoch = tqdm(
        train_dataloader, 
        desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Training',
        miniters=1, ncols=88, position=0,
        leave=True, total=len(train_dataloader), smoothing=.9)

    progress_validation_epoch = tqdm(
        val_dataloader, 
        desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Validation',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(val_dataloader), smoothing=.9)

    train_loss = 0
    train_size = 0
    model.train()
    for idx, (sentence, tags) in enumerate(progress_training_epoch):
        sentence = sentence.cuda()
        tags = tags.cuda()
        model.zero_grad()
        tag_scores = model(sentence)
        loss = loss_function(tag_scores, tags)
        loss.backward()
        optimizer.step()
        train_loss += loss * tags.size()[0]
        train_size += tags.size()[0]

    test_loss = 0
    test_size = 0
    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (sentence, tags) in enumerate(progress_validation_epoch):
            sentence = sentence.cuda()
            tags = tags.cuda()
            tag_scores = model(sentence)
            loss = loss_function(tag_scores, tags)
            predict.append(tag_scores.argmax(dim=1).cpu().numpy())
            target.append(tags.cpu().numpy())        
            test_loss += loss * tags.size()[0]
            test_size += tags.size()[0]
    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)

    print(f'train loss:{train_loss.item()/train_size: .5f}, '
          f'validation loss:{test_loss.item()/test_size: .5f}')

In [ ]:
# Test

progress_test_epoch = tqdm(
    test_dataloader, 
    desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Test',
    miniters=1, ncols=88, position=0, 
    leave=True, total=len(test_dataloader), smoothing=.9)

predict = []
target = []
snrss = []
model.eval()
with torch.no_grad():
    for idx, (sentence, tags) in enumerate(progress_test_epoch):
        sentence = sentence.cuda()
        tags = tags.cuda()
        tag_scores = model(sentence)
        predict.append(tag_scores.argmax(dim=1).cpu().numpy())
        target.append(tags.cpu().numpy())
        snrss.append(sentence.cpu().numpy()[:,0])
        
predict = np.concatenate(predict, axis=0)
target = np.concatenate(target, axis=0)
snrss = np.concatenate(snrss, axis=0)

bsru_scr5_res['bsru_scr5_prd'] = predict
bsru_scr5_res['bsru_scr5_trg'] = target
bsru_scr5_res['bsru_scr5_snr'] = snrss * scr5_std + scr5_mean

# CBSDNN

In [ ]:
# Train
model = CBSDNN()
model.cuda()


loss_function = nn.NLLLoss(weight=torch.tensor([1.0,1.0])).cuda()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

train_dataloader=data.DataLoader(data.TensorDataset(train_x_cnn,train_y),batch_size=TRAIN_BATCH_SIZE, 
                                   shuffle=False, num_workers=16, pin_memory=True)

val_dataloader=data.DataLoader(data.TensorDataset(val_x_cnn,val_y),batch_size=VAL_BATCH_SIZE, 
                                 shuffle=False, num_workers=16, pin_memory=True)

test_dataloader=data.DataLoader(data.TensorDataset(test_x_cnn,test_y), batch_size=VAL_BATCH_SIZE,
                                  shuffle=False, num_workers=16, pin_memory=True)

for epoch_idx in range(NUM_EPOCHS):  # again, normally you would NOT do 300 epochs, it is toy data

    progress_training_epoch = tqdm(
        train_dataloader, 
        desc=f'Epoch {epoch_idx}/{NUM_EPOCHS}, Training',
        miniters=1, ncols=88, position=0,
        leave=True, total=len(train_dataloader), smoothing=.9)

    progress_validation_epoch = tqdm(
        val_dataloader, 
        desc=f'Epoch {epoch_idx}/{NUM_EPOCHS}, Validation',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(val_dataloader), smoothing=.9)

    train_loss = 0
    train_size = 0
    model.train()

    for idx, (input, target) in enumerate(progress_training_epoch):
        input = input.cuda()
        target = target.cuda()
        model.zero_grad()
        predict = model(input)
        loss = loss_function(predict, target)
        loss.backward()
        optimizer.step()
        train_loss += loss * target.size()[0]
        train_size += target.size()[0]

    test_loss = 0
    test_size = 0
    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (batch_input, batch_target) in enumerate(progress_validation_epoch):
            batch_input = batch_input.cuda()
            batch_target = batch_target.cuda()
            batch_predict = model(batch_input)
            loss = loss_function(batch_predict, batch_target)
            predict.append(batch_predict.argmax(dim=1).cpu().numpy())
            target.append(batch_target.cpu().numpy())        
            test_loss += loss * batch_target.size()[0]
            test_size += batch_target.size()[0]
    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)
  
    print(f'train loss:{train_loss.item()/train_size: .5f}, '
          f'validation loss:{test_loss.item()/test_size: .5f}')

In [ ]:
# Test

progress_test_epoch = tqdm(
    test_dataloader, 
    desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Test',
    miniters=1, ncols=88, position=0, 
    leave=True, total=len(test_dataloader), smoothing=.9)

predict = []
target = []
snrss = []
model.eval()
with torch.no_grad():
    for idx, (batch_input, batch_target) in enumerate(progress_test_epoch):
        batch_input = batch_input.cuda()
        batch_target = batch_target.cuda()
        batch_predict = model(batch_input)
        predict.append(batch_predict.argmax(dim=1).cpu().numpy())
        target.append(batch_target.cpu().numpy())        
        snrss.append(batch_input.cpu().numpy()[:,0][:,0])
predict = np.concatenate(predict, axis=0)
target = np.concatenate(target, axis=0)
snrss = np.concatenate(snrss, axis=0)

cbsdnn_scr5_res['cbsdnn_scr5_prd'] = predict
cbsdnn_scr5_res['cbsdnn_scr5_trg'] = target
cbsdnn_scr5_res['cbsdnn_scr5_snr'] = snrss * scr5_std + scr5_mean

# With Pilot

# CNN

In [ ]:
# Train

model = CNN()
model.cuda()

loss_function = nn.CrossEntropyLoss(weight=torch.tensor([1.0,1.0])).cuda()
optimizer = optim.SGD(model.parameters(), lr = 0.01)

train_dataloader=data.DataLoader(data.TensorDataset(train_x_plt_cnn,train_y_plt),batch_size=TRAIN_BATCH_SIZE, 
                                   shuffle=True, num_workers=16, pin_memory=True)

val_dataloader=data.DataLoader(data.TensorDataset(val_x_plt_cnn,val_y_plt),batch_size=VAL_BATCH_SIZE, 
                                 shuffle=False, num_workers=16, pin_memory=True)

test_dataloader=data.DataLoader(data.TensorDataset(test_x_plt_cnn,test_y_plt), batch_size=VAL_BATCH_SIZE,
                                  shuffle=False, num_workers=16, pin_memory=True)

model.train()
start_time = time.time()

for epoch_idx in range(NUM_EPOCHS):  # again, normally you would NOT do 300 epochs, it is toy data

    progress_training_epoch = tqdm(
        train_dataloader, 
        desc=f'Epoch {epoch_idx + 1}/{NUM_EPOCHS}, Training',
        miniters=1, ncols=88, position=0,
        leave=True, total=len(train_dataloader), smoothing=.9)

    progress_validation_epoch = tqdm(
        val_dataloader, 
        desc=f'Epoch {epoch_idx + 1}/{NUM_EPOCHS}, Validation',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(val_dataloader), smoothing=.9)

    train_loss = 0
    train_size = 0
    for idx, (input, target) in enumerate(progress_training_epoch):
        input = input.cuda()
        target = target.cuda()
        model.zero_grad()
        predict = model(input)
        loss = loss_function(predict, target)
        loss.backward()
        optimizer.step()
        train_loss += loss * target.size()[0]
        train_size += target.size()[0]

    test_loss = 0
    test_size = 0
    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (batch_input, batch_target) in enumerate(progress_validation_epoch):
            batch_input = batch_input.cuda()
            batch_target = batch_target.cuda()
            batch_predict = model(batch_input)
            loss = loss_function(batch_predict, batch_target)
            predict.append(batch_predict.argmax(dim=1).cpu().numpy())
            target.append(batch_target.cpu().numpy())        
            test_loss += loss * batch_target.size()[0]
            test_size += batch_target.size()[0]
    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)
   
    print(f'train loss:{train_loss.item()/train_size: .5f}, '
          f'validation loss:{test_loss.item()/test_size: .5f}')
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))

print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

In [ ]:
# Test

progress_test_epoch = tqdm(
    test_dataloader, 
    desc=f'Epoch {epoch_idx + 1}/{NUM_EPOCHS}, Test',
    miniters=1, ncols=88, position=0, 
    leave=True, total=len(test_dataloader), smoothing=.9)

predict = [] # Predicted Label
target = [] # Target Lable
snrss = [] # SNR (Calculated from Noise Variance)
model.eval()
with torch.no_grad():
    for idx, (batch_input, batch_target) in enumerate(progress_test_epoch):
        batch_input = batch_input.cuda()
        batch_target = batch_target.cuda()
        batch_predict = model(batch_input)
        predict.append(batch_predict.argmax(dim=1).cpu().numpy())
        target.append(batch_target.cpu().numpy())        
        snrss.append(batch_input.cpu().numpy()[:,0][:,0])
predict = np.concatenate(predict, axis=0)
target = np.concatenate(target, axis=0)
snrss = np.concatenate(snrss, axis=0)

cnn_scr5_res['cnn_scr5_plt_prd'] = predict
cnn_scr5_res['cnn_scr5_plt_trg'] = target
cnn_scr5_res['cnn_scr5_plt_snr'] = snrss * scr5_std + scr5_mean

# GRU

In [ ]:
# Train

model = GRUTagger(EMBEDDING_DIM, HIDDEN_DIM, TAGSET_SIZE).cuda()
model.cuda()
loss_function = nn.NLLLoss(weight=torch.tensor([1.0,1.0])).cuda()
optimizer = optim.Adam(model.parameters(), 0.0001)

train_dataloader=data.DataLoader(data.TensorDataset(train_x_plt,train_y_plt),batch_size=TRAIN_BATCH_SIZE, 
                                   shuffle=False, num_workers=16, pin_memory=True)

val_dataloader=data.DataLoader(data.TensorDataset(val_x_plt,val_y_plt),batch_size=VAL_BATCH_SIZE, 
                                 shuffle=False, num_workers=16, pin_memory=True)

test_dataloader=data.DataLoader(data.TensorDataset(test_x_plt,test_y_plt), batch_size=VAL_BATCH_SIZE,
                                  shuffle=False, num_workers=16, pin_memory=True)

for epoch_idx in range(NUM_EPOCHS): 

    progress_training_epoch = tqdm(
        train_dataloader, 
        desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Training',
        miniters=1, ncols=88, position=0,
        leave=True, total=len(train_dataloader), smoothing=.9)

    progress_validation_epoch = tqdm(
        val_dataloader, 
        desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Validation',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(val_dataloader), smoothing=.9)

    train_loss = 0
    train_size = 0
    model.train()
    for idx, (sentence, tags) in enumerate(progress_training_epoch):
        sentence = sentence.cuda()
        tags = tags.cuda()
        model.zero_grad()
        tag_scores = model(sentence)
        loss = loss_function(tag_scores, tags)
        loss.backward()
        optimizer.step()
        train_loss += loss * tags.size()[0]
        train_size += tags.size()[0]

    test_loss = 0
    test_size = 0
    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (sentence, tags) in enumerate(progress_validation_epoch):
            sentence = sentence.cuda()
            tags = tags.cuda()
            tag_scores = model(sentence)
            loss = loss_function(tag_scores, tags)
            predict.append(tag_scores.argmax(dim=1).cpu().numpy())
            target.append(tags.cpu().numpy())        
            test_loss += loss * tags.size()[0]
            test_size += tags.size()[0]
    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)

    print(f'train loss:{train_loss.item()/train_size: .5f}, '
          f'validation loss:{test_loss.item()/test_size: .5f}')

In [ ]:
# Test

progress_test_epoch = tqdm(
    test_dataloader, 
    desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Test',
    miniters=1, ncols=88, position=0, 
    leave=True, total=len(test_dataloader), smoothing=.9)

predict = []
target = []
snrss = []
model.eval()
with torch.no_grad():
    for idx, (sentence, tags) in enumerate(progress_test_epoch):
        sentence = sentence.cuda()
        tags = tags.cuda()
        tag_scores = model(sentence)
        predict.append(tag_scores.argmax(dim=1).cpu().numpy())
        target.append(tags.cpu().numpy())
        snrss.append(sentence.cpu().numpy()[:,0])
        
predict = np.concatenate(predict, axis=0)
target = np.concatenate(target, axis=0)
snrss = np.concatenate(snrss, axis=0)

gru_scr5_res['gru_scr5_plt_prd'] = predict
gru_scr5_res['gru_scr5_plt_trg'] = target
gru_scr5_res['gru_scr5_plt_snr'] = snrss * scr5_std + scr5_mean

# BGRU

In [ ]:
# Train

model = BGRUTagger(EMBEDDING_DIM, HIDDEN_DIM, TAGSET_SIZE).cuda()
model.cuda()
loss_function = nn.NLLLoss(weight=torch.tensor([1.0,1.0])).cuda()
optimizer = optim.Adam(model.parameters(), 0.0001)

train_dataloader=data.DataLoader(data.TensorDataset(train_x_plt,train_y_plt),batch_size=TRAIN_BATCH_SIZE, 
                                   shuffle=False, num_workers=16, pin_memory=True)

val_dataloader=data.DataLoader(data.TensorDataset(val_x_plt,val_y_plt),batch_size=VAL_BATCH_SIZE, 
                                 shuffle=False, num_workers=16, pin_memory=True)

test_dataloader=data.DataLoader(data.TensorDataset(test_x_plt,test_y_plt), batch_size=VAL_BATCH_SIZE,
                                  shuffle=False, num_workers=16, pin_memory=True)

for epoch_idx in range(NUM_EPOCHS): 

    progress_training_epoch = tqdm(
        train_dataloader, 
        desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Training',
        miniters=1, ncols=88, position=0,
        leave=True, total=len(train_dataloader), smoothing=.9)

    progress_validation_epoch = tqdm(
        val_dataloader, 
        desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Validation',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(val_dataloader), smoothing=.9)

    train_loss = 0
    train_size = 0
    model.train()
    for idx, (sentence, tags) in enumerate(progress_training_epoch):
        sentence = sentence.cuda()
        tags = tags.cuda()
        model.zero_grad()
        tag_scores = model(sentence)
        loss = loss_function(tag_scores, tags)
        loss.backward()
        optimizer.step()
        train_loss += loss * tags.size()[0]
        train_size += tags.size()[0]

    test_loss = 0
    test_size = 0
    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (sentence, tags) in enumerate(progress_validation_epoch):
            sentence = sentence.cuda()
            tags = tags.cuda()
            tag_scores = model(sentence)
            loss = loss_function(tag_scores, tags)
            predict.append(tag_scores.argmax(dim=1).cpu().numpy())
            target.append(tags.cpu().numpy())        
            test_loss += loss * tags.size()[0]
            test_size += tags.size()[0]
    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)

    print(f'train loss:{train_loss.item()/train_size: .5f}, '
          f'validation loss:{test_loss.item()/test_size: .5f}')

In [ ]:
# Test

progress_test_epoch = tqdm(
    test_dataloader, 
    desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Test',
    miniters=1, ncols=88, position=0, 
    leave=True, total=len(test_dataloader), smoothing=.9)

predict = []
target = []
snrss = []
model.eval()
with torch.no_grad():
    for idx, (sentence, tags) in enumerate(progress_test_epoch):
        sentence = sentence.cuda()
        tags = tags.cuda()
        tag_scores = model(sentence)
        predict.append(tag_scores.argmax(dim=1).cpu().numpy())
        target.append(tags.cpu().numpy())
        snrss.append(sentence.cpu().numpy()[:,0])
        
predict = np.concatenate(predict, axis=0)
target = np.concatenate(target, axis=0)
snrss = np.concatenate(snrss, axis=0)

bgru_scr5_res['bgru_scr5_plt_prd'] = predict
bgru_scr5_res['bgru_scr5_plt_trg'] = target
bgru_scr5_res['bgru_scr5_plt_snr'] = snrss * scr5_std + scr5_mean

# BSRU

In [ ]:
# Train

model = BSRUTagger(EMBEDDING_DIM, HIDDEN_DIM, TAGSET_SIZE).cuda()
model.cuda()
loss_function = nn.NLLLoss(weight=torch.tensor([1.0,1.0])).cuda()
optimizer = optim.Adam(model.parameters(), 0.0001)

train_dataloader=data.DataLoader(data.TensorDataset(train_x_plt,train_y_plt),batch_size=TRAIN_BATCH_SIZE, 
                                   shuffle=False, num_workers=16, pin_memory=True)

val_dataloader=data.DataLoader(data.TensorDataset(val_x_plt,val_y_plt),batch_size=VAL_BATCH_SIZE, 
                                 shuffle=False, num_workers=16, pin_memory=True)

test_dataloader=data.DataLoader(data.TensorDataset(test_x_plt,test_y_plt), batch_size=VAL_BATCH_SIZE,
                                  shuffle=False, num_workers=16, pin_memory=True)

for epoch_idx in range(NUM_EPOCHS): 

    progress_training_epoch = tqdm(
        train_dataloader, 
        desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Training',
        miniters=1, ncols=88, position=0,
        leave=True, total=len(train_dataloader), smoothing=.9)

    progress_validation_epoch = tqdm(
        val_dataloader, 
        desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Validation',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(val_dataloader), smoothing=.9)

    train_loss = 0
    train_size = 0
    model.train()
    for idx, (sentence, tags) in enumerate(progress_training_epoch):
        sentence = sentence.cuda()
        tags = tags.cuda()
        model.zero_grad()
        tag_scores = model(sentence)
        loss = loss_function(tag_scores, tags)
        loss.backward()
        optimizer.step()
        train_loss += loss * tags.size()[0]
        train_size += tags.size()[0]

    test_loss = 0
    test_size = 0
    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (sentence, tags) in enumerate(progress_validation_epoch):
            sentence = sentence.cuda()
            tags = tags.cuda()
            tag_scores = model(sentence)
            loss = loss_function(tag_scores, tags)
            predict.append(tag_scores.argmax(dim=1).cpu().numpy())
            target.append(tags.cpu().numpy())        
            test_loss += loss * tags.size()[0]
            test_size += tags.size()[0]
    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)

    print(f'train loss:{train_loss.item()/train_size: .5f}, '
          f'validation loss:{test_loss.item()/test_size: .5f}')

In [ ]:
# Test

progress_test_epoch = tqdm(
    test_dataloader, 
    desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Test',
    miniters=1, ncols=88, position=0, 
    leave=True, total=len(test_dataloader), smoothing=.9)

predict = []
target = []
snrss = []
model.eval()
with torch.no_grad():
    for idx, (sentence, tags) in enumerate(progress_test_epoch):
        sentence = sentence.cuda()
        tags = tags.cuda()
        tag_scores = model(sentence)
        predict.append(tag_scores.argmax(dim=1).cpu().numpy())
        target.append(tags.cpu().numpy())
        snrss.append(sentence.cpu().numpy()[:,0])
        
predict = np.concatenate(predict, axis=0)
target = np.concatenate(target, axis=0)
snrss = np.concatenate(snrss, axis=0)

bsru_scr5_res['bsru_scr5_plt_prd'] = predict
bsru_scr5_res['bsru_scr5_plt_trg'] = target
bsru_scr5_res['bsru_scr5_plt_snr'] = snrss * scr5_std + scr5_mean

# CBSDNN

In [ ]:
# Train
model = CBSDNN()
model.cuda()


loss_function = nn.NLLLoss(weight=torch.tensor([1.0,1.0])).cuda()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

train_dataloader=data.DataLoader(data.TensorDataset(train_x_plt_cnn,train_y_plt),batch_size=TRAIN_BATCH_SIZE, 
                                   shuffle=False, num_workers=16, pin_memory=True)

val_dataloader=data.DataLoader(data.TensorDataset(val_x_plt_cnn,val_y_plt),batch_size=VAL_BATCH_SIZE, 
                                 shuffle=False, num_workers=16, pin_memory=True)

test_dataloader=data.DataLoader(data.TensorDataset(test_x_plt_cnn,test_y_plt), batch_size=VAL_BATCH_SIZE,
                                  shuffle=False, num_workers=16, pin_memory=True)

start_time = time.time()

for epoch_idx in range(NUM_EPOCHS):  # again, normally you would NOT do 300 epochs, it is toy data

    progress_training_epoch = tqdm(
        train_dataloader, 
        desc=f'Epoch {epoch_idx}/{NUM_EPOCHS}, Training',
        miniters=1, ncols=88, position=0,
        leave=True, total=len(train_dataloader), smoothing=.9)

    progress_validation_epoch = tqdm(
        val_dataloader, 
        desc=f'Epoch {epoch_idx}/{NUM_EPOCHS}, Validation',
        miniters=1, ncols=88, position=0, 
        leave=True, total=len(val_dataloader), smoothing=.9)

    train_loss = 0
    train_size = 0
    model.train()

    for idx, (input, target) in enumerate(progress_training_epoch):
        input = input.cuda()
        target = target.cuda()
        model.zero_grad()
        predict = model(input)
        loss = loss_function(predict, target)
        loss.backward()
        optimizer.step()
        train_loss += loss * target.size()[0]
        train_size += target.size()[0]

    test_loss = 0
    test_size = 0
    predict = []
    target = []
    model.eval()
    with torch.no_grad():
        for idx, (batch_input, batch_target) in enumerate(progress_validation_epoch):
            batch_input = batch_input.cuda()
            batch_target = batch_target.cuda()
            batch_predict = model(batch_input)
            loss = loss_function(batch_predict, batch_target)
            predict.append(batch_predict.argmax(dim=1).cpu().numpy())
            target.append(batch_target.cpu().numpy())        
            test_loss += loss * batch_target.size()[0]
            test_size += batch_target.size()[0]
    predict = np.concatenate(predict, axis=0)
    target = np.concatenate(target, axis=0)
  
    print(f'train loss:{train_loss.item()/train_size: .5f}, '
          f'validation loss:{test_loss.item()/test_size: .5f}')

In [ ]:
# Test

progress_test_epoch = tqdm(
    test_dataloader, 
    desc=f'Epoch {epoch_idx+1}/{NUM_EPOCHS}, Test',
    miniters=1, ncols=88, position=0, 
    leave=True, total=len(test_dataloader), smoothing=.9)

predict = []
target = []
snrss = []
model.eval()
with torch.no_grad():
    for idx, (batch_input, batch_target) in enumerate(progress_test_epoch):
        batch_input = batch_input.cuda()
        batch_target = batch_target.cuda()
        batch_predict = model(batch_input)
        predict.append(batch_predict.argmax(dim=1).cpu().numpy())
        target.append(batch_target.cpu().numpy())        
        snrss.append(batch_input.cpu().numpy()[:,0][:,0])
predict = np.concatenate(predict, axis=0)
target = np.concatenate(target, axis=0)
snrss = np.concatenate(snrss, axis=0)

cbsdnn_scr5_res['cbsdnn_scr5_plt_prd'] = predict
cbsdnn_scr5_res['cbsdnn_scr5_plt_trg'] = target
cbsdnn_scr5_res['cbsdnn_scr5_plt_snr'] = snrss * scr5_std + scr5_mean

# Save Output Files

In [ ]:
# CNN
outfile = os.path.join(os.getcwd(), 'cnn_scr5_res.pickle')
if os.path.exists(outfile):
        os.replace(outfile, outfile + ".old")
        
with open(outfile, 'wb') as file:
    pickle.dump(cnn_scr5_res, file)
    
# GRU
outfile = os.path.join(os.getcwd(), 'gru_scr5_res.pickle')
if os.path.exists(outfile):
        os.replace(outfile, outfile + ".old")
        
with open(outfile, 'wb') as file:
    pickle.dump(gru_scr5_res, file)
    
# BGRU
outfile = os.path.join(os.getcwd(), 'bgru_scr5_res.pickle')
if os.path.exists(outfile):
        os.replace(outfile, outfile + ".old")
        
with open(outfile, 'wb') as file:
    pickle.dump(bgru_scr5_res, file)
    
# BSRU
outfile = os.path.join(os.getcwd(), 'bsru_scr5_res.pickle')
if os.path.exists(outfile):
        os.replace(outfile, outfile + ".old")
        
with open(outfile, 'wb') as file:
    pickle.dump(bsru_scr5_res, file)

# CBSDNN
outfile = os.path.join(os.getcwd(), 'cbsdnn_scr5_res.pickle')
if os.path.exists(outfile):
        os.replace(outfile, outfile + ".old")
        
with open(outfile, 'wb') as file:
    pickle.dump(cbsdnn_scr5_res, file)